# *Rice - Classification Project*


*Bu projede pirinç türlerini algılayan bir **CNN Modeli** geliştireceğiz. Modeli kaydedip Streamlit uygulamasına çevireceğiz ve HugginFace de çalışır hale getireceğiz.*

### *İmport Libraries*

In [82]:
import os                  # Dosya ve dizin işlemleri için
import random              # Rastgele seçimler yapmak için
import warnings            # Uyarı mesajlarını yönetmek için
warnings.filterwarnings('ignore') # Tüm uyarı mesajlarını görmezden gelmek için filterwarnings ile 'ignore' ayarını yapıyoruz

import numpy as np         # Sayısal hesaplamalar ve matris işlemleri
import pandas as pd        # Veri analizi ve tablo işlemleri
import cv2                 # Görüntü işleme ve video/frame işlemleri
import kagglehub           # Kaggle veri setlerini kolayca çekmek için

from sklearn.model_selection import train_test_split  # Veriyi eğitim/test setlerine ayırmak için

from tensorflow.keras.models import Model            # Keras Functional API model tanımı
from tensorflow.keras.layers import Flatten, Dense   # Katmanlar: Flatten ve tam bağlantılı Dense
from tensorflow.keras.applications import VGG16      # Önceden eğitilmiş VGG16 modeli
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Görüntü artırma ve ön işleme


### *İmport Data*

In [4]:
# Sınıf etiketlerini bir liste halinde tanımlıyoruz
# 'Arborio' -> Arborio örnekleri
# 'Basmati' -> Basmati örnekleri
# 'Ipsala' -> Ipsala örnekleri
# 'Jasmine' -> Jasmine örnekleri
# 'Karacadag' -> Karacadag örnekleri
labels = ['Arborio', 'Basmati', 'Ipsala', 'Jasmine', 'Karacadag']

# Görüntülerin bulunduğu ana dizin yolunu belirtiyoruz
# Download latest version
path = kagglehub.dataset_download("muratkokludataset/rice-image-dataset")

print("Path to dataset files:", path)

100%|██████████| 219M/219M [00:10<00:00, 21.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/muratkokludataset/rice-image-dataset/versions/1


In [5]:
img_path = os.path.join(path, "Rice_Image_Dataset/")
os.listdir(img_path)

['Ipsala',
 'Arborio',
 'Jasmine',
 'Karacadag',
 'Basmati',
 'Rice_Citation_Request.txt']

### *CNN with Transfer Learning*

In [27]:


# Modelin girdi boyutu
img_width, img_height = 224, 224

# --- Veri hazırlama ---
train_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.20)

train_datagenerator = train_datagen.flow_from_directory(
    directory=img_path,
    target_size=(img_width, img_height),
    class_mode='categorical',
    subset='training'
)

test_datagenerator = train_datagen.flow_from_directory(
    directory=img_path,
    target_size=(img_width, img_height),
    class_mode='categorical',
    subset='validation'
)

# --- Transfer Learning: VGG16 base ---
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.trainable = False  # Tüm katmanları dondur

# --- Functional API ile üst katmanlar ---
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(5, activation='softmax')(x)  # 5 sınıf

# --- Model tanımı ---
model = Model(inputs=base_model.input, outputs=outputs)

# --- Compile ---
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# --- Eğitim ---
model.fit(
    train_datagenerator,
    epochs=10,
    validation_data=test_datagenerator
)



Found 60000 images belonging to 5 classes.
Found 15000 images belonging to 5 classes.
Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 409s 217ms/step - accuracy: 0.9395 - loss: 0.4100 - val_accuracy: 0.9780 - val_loss: 0.0680
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 404s 215ms/step - accuracy: 0.9828 - loss: 0.0491 - val_accuracy: 0.9911 - val_loss: 0.0283
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 404s 215ms/step - accuracy: 0.9888 - loss: 0.0332 - val_accuracy: 0.9925 - val_loss: 0.0234
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 404s 216ms/step - accuracy: 0.9920 - loss: 0.0244 - val_accuracy: 0.9901 - val_loss: 0.0312
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 404s 215ms/step - accuracy: 0.9927 - loss: 0.0223 - val_accuracy: 0.9961 - val_loss: 0.0143
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 404s 216ms/step - accuracy: 0.9935 - loss: 0.0198 - val_accuracy: 0.9952 - val_loss: 0.0172
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 404s 216ms/step - accuracy: 0.9947 - loss: 0.0154 - val_accuracy: 0.

#### *Save the Learning Model*

In [28]:
# --- Kaydet (Keras 3 uyumlu) ---
model.save("rice_model_TL.keras")  # Streamlit ve Hugging Face uyumlu


*Bu projede pirinç türlerini algılayan bir CNN Modeli geliştirdik.* <br>
*Datayı kaggle üzerinden çektik.* <br>
*Kendi datamızı önceden eğitilmiş **VGG16** modeli ile birleştirip geliştirdik.* <br>
*%99.66 başarı elde ettik.*